In [2]:
import numpy as np
import nltk 
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as st


In [6]:
from pycorenlp import StanfordCoreNLP
#MUST RUN 
# Run the server using all jars in the current directory (e.g., the CoreNLP home directory)
#java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000
nlp = StanfordCoreNLP('http://localhost:9000')

In [19]:
text = "'the quick brown fox jumps over the lazy dog.'"
output = nlp.annotate(text, properties={
    'annotators': 'openie',
    'outputFormat': 'json'
    })

In [20]:
output

{u'sentences': [{u'basicDependencies': [{u'dep': u'ROOT',
     u'dependent': 6,
     u'dependentGloss': u'jumps',
     u'governor': 0,
     u'governorGloss': u'ROOT'},
    {u'dep': u'punct',
     u'dependent': 1,
     u'dependentGloss': u'`',
     u'governor': 6,
     u'governorGloss': u'jumps'},
    {u'dep': u'det',
     u'dependent': 2,
     u'dependentGloss': u'the',
     u'governor': 5,
     u'governorGloss': u'fox'},
    {u'dep': u'amod',
     u'dependent': 3,
     u'dependentGloss': u'quick',
     u'governor': 5,
     u'governorGloss': u'fox'},
    {u'dep': u'amod',
     u'dependent': 4,
     u'dependentGloss': u'brown',
     u'governor': 5,
     u'governorGloss': u'fox'},
    {u'dep': u'nsubj',
     u'dependent': 5,
     u'dependentGloss': u'fox',
     u'governor': 6,
     u'governorGloss': u'jumps'},
    {u'dep': u'case',
     u'dependent': 7,
     u'dependentGloss': u'over',
     u'governor': 10,
     u'governorGloss': u'dog'},
    {u'dep': u'det',
     u'dependent': 8,
     u

In [79]:
news_csv = pd.read_csv("news_reuters_10.csv", error_bad_lines=False, header = None, names = ["stock", "company", "date", "title", "summary", "type", "website"])
google_price_csv = pd.read_csv("GOOG_2006-01-01_to_2017-11-01.csv")

In [4]:
number_to_month = {"01": "Jan", "02":"Feb", "03":"Mar", "04":"Apr", "05":"May", "06": "Jun", "07":"Jul", "08":"Aug", "09":"Sep", "10":"Oct", "11":"Nov", "12":"Dec"}
def conv_num_to_string(d):
    year = d[0:4]
    month = d[4:6]
    day = d[6:8]
    new = day + "-" + number_to_month[month] + "-" + year[2:4]
    return new 

In [102]:
def sentiment_to_price_plot(stock, day_lag, pos_or_neg): #ex: sentiment_to_price_plot("AAPL", 1, 'neg')
    stock_data = news_csv[news_csv["stock"] == stock]
    stock_price_csv = pd.read_csv(stock+"_2006-01-01_to_2017-11-01.csv")
    temp_x = []
    temp_y = []
    for index, row in stock_data.iterrows():
        ss = sid.polarity_scores(row["summary"])
        score = ss[pos_or_neg]

        day = conv_num_to_string(str(row["date"]) )

        if day in stock_price_csv["Date"].values:

            temp_x.append(score)

            row_index = stock_price_csv.index[stock_price_csv["Date"] == day].tolist()[0]
            next_price = stock_price_csv.iloc[row_index - day_lag  ]
            #print next_price["Date"], google_price_csv.iloc[row_index]["Date"]
            diff = next_price["Close"] - next_price["Open"]
            temp_y.append(diff)
    print st.spearmanr(temp_x, temp_y)
    plt.plot(temp_x, temp_y, "o")
    plt.ylabel("Closing Minus Opening after" + str(day_lag) + "Days")
    plt.xlabel(pos_or_neg + "NLTK Vader-Sentiment Score of Current Day's Summary")
    plt.title(stock + ",  " + str(day_lag) + " Day Lag,  " + pos_or_neg)
    plt.show()
    